In [1]:
import pandas as pd
import numpy as np

from utils.load_data_for_eda import load_data
from utils.effect_sizes import compute_effect_sizes_by_group
df = load_data('data/commodity_prices.csv')
df = compute_effect_sizes_by_group(df)

In [2]:
df.head(10)

,Product_Type,Arrival_Date,Market,Is_VFPCK,Season,Year,Modal_Price,Max_Price,Min_Price,eta2_Season,omega2_Season,eta2_Market,omega2_Market,eta2_Year,omega2_Year
0,Alsandikai|Alsandikai|FAQ,2023-12-13,North Paravur,False,Winter,2023,5200.0,6000.0,5000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
1,Alsandikai|Alsandikai|FAQ,2023-12-14,North Paravur,False,Winter,2023,6200.0,6500.0,6000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
2,Alsandikai|Alsandikai|FAQ,2023-12-16,North Paravur,False,Winter,2023,4800.0,5600.0,4600.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
3,Alsandikai|Alsandikai|FAQ,2023-12-18,North Paravur,False,Winter,2023,3500.0,4500.0,3000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
4,Alsandikai|Alsandikai|FAQ,2023-12-19,North Paravur,False,Winter,2023,5500.0,6000.0,5500.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
5,Alsandikai|Alsandikai|FAQ,2023-12-20,North Paravur,False,Winter,2023,5500.0,6000.0,5000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
6,Alsandikai|Alsandikai|FAQ,2023-12-21,North Paravur,False,Winter,2023,4500.0,5600.0,4000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
7,Alsandikai|Alsandikai|FAQ,2023-12-22,North Paravur,False,Winter,2023,4500.0,5600.0,4000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
8,Alsandikai|Alsandikai|FAQ,2023-12-23,North Paravur,False,Winter,2023,5500.0,6000.0,5000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769
9,Alsandikai|Alsandikai|FAQ,2023-12-26,North Paravur,False,Winter,2023,3400.0,5600.0,3000.0,0.386076,0.38163,0.0,0.0,0.016996,0.012769


In [87]:
df['eta2_Market'].nunique(), df['omega2_Market'].nunique(), df['eta2_Season'].nunique(), df['omega2_Season'].nunique(), df['eta2_Year'].nunique(), df['omega2_Year'].nunique()

(98, 98, 124, 124, 112, 112)

In [86]:
df['Product_Type'].nunique()

124

In [ ]:

for product, group in df.groupby('Product_Type'):
    group['log_Modal_Price'] = group["Modal_Price"].apply(lambda X: np.log(X) if X > 0 else np.nan)
    prices = [grp['log_Modal_Price'].values for _, grp in group.groupby('Season')]
    mean_prices = [np.mean(p) for p in prices]
    grand_mean = np.mean(np.concatenate(prices))
    size_prices = [len(p) for p in prices]
    variance_prices = [np.var(p, ddof=1) if len(p) > 1 else 0 for p in prices]
    SS_between = sum([(mean_prices[i] - grand_mean)**2 * size_prices[i] for i in range(len(prices))])
    SS_within = sum([(size_prices[i] - 1) * variance_prices[i] for i in range(len(prices))])
    SS_total = SS_between + SS_within
    eta2_season = SS_between / SS_total if SS_total > 0 else 0
    df_between = len(prices) - 1
    df_within = sum(size_prices) - len(prices)
    MS_within = SS_within / df_within if df_within > 0 else 0
    omega2_season = (SS_between - df_between * MS_within) / (SS_total + MS_within) if (SS_total + MS_within) > 0 else 0
    df['eta2_season'] = eta2_season
    df['omega2_season'] = omega2_season
# print(prices)
# # print(prices)
# # print(grand_mean)
# # print(mean_prices)
# # print(variance_prices)
# print(eta2_season, omega2_season)

In [67]:

for product, group in df.groupby('Product_Type'):
    group['log_Modal_Price'] = group["Modal_Price"].apply(lambda X: np.log(X) if X > 0 else np.nan)
    prices = [grp['log_Modal_Price'].values for _, grp in group.groupby('Market')]
    mean_prices = [np.mean(p) for p in prices]
    grand_mean = np.mean(np.concatenate(prices))
    size_prices = [len(p) for p in prices]
    variance_prices = [np.var(p, ddof=1) if len(p) > 1 else 0 for p in prices]
    SS_between = sum([(mean_prices[i] - grand_mean)**2 * size_prices[i] for i in range(len(prices))])
    SS_within = sum([(size_prices[i] - 1) * variance_prices[i] for i in range(len(prices))])
    SS_total = SS_between + SS_within
    eta2_market = SS_between / SS_total if SS_total > 0 else 0
    df_between = len(prices) - 1
    df_within = sum(size_prices) - len(prices)
    MS_within = SS_within / df_within if df_within > 0 else 0
    omega2_market = (SS_between - df_between * MS_within) / (SS_total + MS_within) if (SS_total + MS_within) > 0 else 0
    df['eta2_season'] = eta2_market
    df['omega2_season'] = omega2_market
print(prices)
# # print(prices)
print(grand_mean)
print(mean_prices)
print(variance_prices)
print(SS_between, SS_within, SS_total)
print(df_between, df_within, MS_within)
print(eta2_market, omega2_market)
print()

[array([8.00636757, 7.82404601, 7.82404601, 7.82404601, 7.82404601,
       7.82404601, 8.51719319, 7.82404601, 8.51719319, 8.51719319,
       8.51719319, 7.82404601, 7.82404601, 8.00636757, 8.00636757,
       8.00636757, 8.00636757, 7.82404601, 7.82404601, 7.82404601,
       7.82404601, 7.82404601, 7.82404601, 8.00636757, 7.82404601,
       7.82404601, 8.00636757, 7.82404601, 7.82404601, 7.82404601,
       7.82404601, 7.82404601, 7.82404601, 8.16051825, 7.82404601,
       8.16051825, 8.16051825, 7.82404601, 7.82404601, 8.16051825,
       8.16051825, 8.00636757, 8.16051825, 8.16051825, 7.82404601,
       7.82404601, 8.16051825, 7.82404601, 8.16051825, 8.00636757,
       7.82404601, 8.16051825, 7.60090246, 7.60090246, 7.60090246,
       7.60090246, 8.00636757, 7.60090246, 7.60090246, 7.60090246,
       7.82404601, 7.60090246, 7.37775891, 8.16051825, 7.60090246,
       7.60090246, 7.24422752, 7.60090246, 7.60090246, 7.60090246,
       7.60090246, 7.69621264, 7.60090246, 7.60090246, 7.6009

In [56]:
df.head(10)

,Product_Type,Arrival_Date,Market,Is_VFPCK,Season,Year,Modal_Price,Max_Price,Min_Price,eta2_season,omega2_season
0,Alsandikai|Alsandikai|FAQ,2023-12-13,North Paravur,False,Winter,2023,5200.0,6000.0,5000.0,0.0,0.0
1,Alsandikai|Alsandikai|FAQ,2023-12-14,North Paravur,False,Winter,2023,6200.0,6500.0,6000.0,0.0,0.0
2,Alsandikai|Alsandikai|FAQ,2023-12-16,North Paravur,False,Winter,2023,4800.0,5600.0,4600.0,0.0,0.0
3,Alsandikai|Alsandikai|FAQ,2023-12-18,North Paravur,False,Winter,2023,3500.0,4500.0,3000.0,0.0,0.0
4,Alsandikai|Alsandikai|FAQ,2023-12-19,North Paravur,False,Winter,2023,5500.0,6000.0,5500.0,0.0,0.0
5,Alsandikai|Alsandikai|FAQ,2023-12-20,North Paravur,False,Winter,2023,5500.0,6000.0,5000.0,0.0,0.0
6,Alsandikai|Alsandikai|FAQ,2023-12-21,North Paravur,False,Winter,2023,4500.0,5600.0,4000.0,0.0,0.0
7,Alsandikai|Alsandikai|FAQ,2023-12-22,North Paravur,False,Winter,2023,4500.0,5600.0,4000.0,0.0,0.0
8,Alsandikai|Alsandikai|FAQ,2023-12-23,North Paravur,False,Winter,2023,5500.0,6000.0,5000.0,0.0,0.0
9,Alsandikai|Alsandikai|FAQ,2023-12-26,North Paravur,False,Winter,2023,3400.0,5600.0,3000.0,0.0,0.0


In [61]:
df[eta2_season].nunique()

KeyError: np.float64(0.0)

In [43]:
group.shape

(119, 12)

In [46]:
df[df['Product_Type'] == 'Water Melon|Water Melon|Large']['Season'].shape

(119,)

In [41]:
group.head(10)

,Product_Type,Arrival_Date,Market,Is_VFPCK,Season,Year,Modal_Price,Max_Price,Min_Price,log_Modal_Price,eta2_season,omega2_season
167831,Water Melon|Water Melon|Large,2025-02-07,Piravam,False,Winter,2025,3000.0,4000.0,2000.0,8.006368,0.100681,0.084521
167832,Water Melon|Water Melon|Large,2025-02-09,Piravam,False,Winter,2025,2500.0,3000.0,2000.0,7.824046,0.100681,0.084521
167833,Water Melon|Water Melon|Large,2025-02-10,Piravam,False,Winter,2025,2500.0,3000.0,2000.0,7.824046,0.100681,0.084521
167834,Water Melon|Water Melon|Large,2025-02-12,Piravam,False,Winter,2025,2500.0,3000.0,2000.0,7.824046,0.100681,0.084521
167835,Water Melon|Water Melon|Large,2025-02-13,Piravam,False,Winter,2025,2500.0,3000.0,2000.0,7.824046,0.100681,0.084521
167836,Water Melon|Water Melon|Large,2025-02-14,Piravam,False,Winter,2025,2500.0,3000.0,2000.0,7.824046,0.100681,0.084521
167837,Water Melon|Water Melon|Large,2025-02-16,Piravam,False,Winter,2025,5000.0,6000.0,4000.0,8.517193,0.100681,0.084521
167838,Water Melon|Water Melon|Large,2025-02-17,Piravam,False,Winter,2025,2500.0,3000.0,2000.0,7.824046,0.100681,0.084521
167839,Water Melon|Water Melon|Large,2025-02-21,Piravam,False,Winter,2025,5000.0,6000.0,4000.0,8.517193,0.100681,0.084521
167840,Water Melon|Water Melon|Large,2025-02-23,Piravam,False,Winter,2025,5000.0,6000.0,4000.0,8.517193,0.100681,0.084521


In [48]:
df.head(10)

,Product_Type,Arrival_Date,Market,Is_VFPCK,Season,Year,Modal_Price,Max_Price,Min_Price,log_Modal_Price,eta2_season,omega2_season
0,Alsandikai|Alsandikai|FAQ,2023-12-13,North Paravur,False,Winter,2023,5200.0,6000.0,5000.0,8.556414,0.100681,0.084521
1,Alsandikai|Alsandikai|FAQ,2023-12-14,North Paravur,False,Winter,2023,6200.0,6500.0,6000.0,8.732305,0.100681,0.084521
2,Alsandikai|Alsandikai|FAQ,2023-12-16,North Paravur,False,Winter,2023,4800.0,5600.0,4600.0,8.476371,0.100681,0.084521
3,Alsandikai|Alsandikai|FAQ,2023-12-18,North Paravur,False,Winter,2023,3500.0,4500.0,3000.0,8.160518,0.100681,0.084521
4,Alsandikai|Alsandikai|FAQ,2023-12-19,North Paravur,False,Winter,2023,5500.0,6000.0,5500.0,8.612503,0.100681,0.084521
5,Alsandikai|Alsandikai|FAQ,2023-12-20,North Paravur,False,Winter,2023,5500.0,6000.0,5000.0,8.612503,0.100681,0.084521
6,Alsandikai|Alsandikai|FAQ,2023-12-21,North Paravur,False,Winter,2023,4500.0,5600.0,4000.0,8.411833,0.100681,0.084521
7,Alsandikai|Alsandikai|FAQ,2023-12-22,North Paravur,False,Winter,2023,4500.0,5600.0,4000.0,8.411833,0.100681,0.084521
8,Alsandikai|Alsandikai|FAQ,2023-12-23,North Paravur,False,Winter,2023,5500.0,6000.0,5000.0,8.612503,0.100681,0.084521
9,Alsandikai|Alsandikai|FAQ,2023-12-26,North Paravur,False,Winter,2023,3400.0,5600.0,3000.0,8.131531,0.100681,0.084521
